## 日内分钟级数据的CVaR因子

In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
tqdm.pandas()
# pd.set_option('display.max_rows', 50)

/mnt/datadisk2/aglv/anaconda3/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
def cVaR_series(series, c_level=0.9, is_low=True):
    '''
    CVaR:分布的尾部积分, Parameters:
    c_level:p值,默认0.9
    is_low:返回的是否是左侧CVaR,=False时返回右侧,默认True
    '''
    right_VaR = series.quantile(c_level)
    left_VaR = series.quantile(1- c_level)

    if is_low:
        X_low = pd.Series(series[series < left_VaR])
        low_mean = X_low.mean() if len(X_low) > 0 else 0
        return low_mean
    else:
        X_high = pd.Series(series[series > right_VaR])
        high_mean = X_high.mean() if len(X_high) > 0 else 0
        return -high_mean

In [3]:
open2021 = pd.read_hdf('/mnt/datadisk2/aglv/aglv/lab_aglv/open_2021.h5')
open2021 = open2021.set_index(['Date', 'EndTime'])
open2021

InstrumentID      SH600000  SH600004  SH600006  SH600007  SH600008  SH600009  \
Date     EndTime                                                               
20210104 09:31        9.64     14.10      8.95     12.45      2.86     75.70   
         09:32        9.64     14.01      8.82     12.35      2.86     76.46   
         09:33        9.60     13.97      8.84     12.36      2.84     76.17   
         09:34        9.60     14.00      8.83     12.37      2.85     76.10   
         09:35        9.58     13.90      8.83     12.33      2.85     75.74   
...                    ...       ...       ...       ...       ...       ...   
20211231 14:56        8.55     12.04      6.81     14.41      3.41     46.68   
         14:57        8.54     12.03      6.81     14.41      3.40     46.68   
         14:58        8.55     12.03      6.81     14.39      3.41     46.70   
         14:59        8.55     12.03      6.81     14.39      3.41     46.70   
         15:00        8.55     12.03      6.81     14.39      3.41     46.70   

InstrumentID      SH600010  SH600011  SH600012  SH600015  ...  SZ301190  \
Date     EndTime                                          ...             
20210104 09:31        1.17      4.50      6.14      6.22  ...       NaN   
         09:32        1.18      4.49      6.12      6.22  ...       NaN   
         09:33        1.17      4.47      6.16      6.20  ...       NaN   
         09:34        1.17      4.47      6.17      6.21  ...       NaN   
         09:35        1.17      4.46      6.16      6.21  ...       NaN   
...                    ...       ...       ...       ...  ...       ...   
20211231 14:56        2.78      9.64      7.07      5.59  ...     30.31   
         14:57        2.79      9.65      7.07      5.59  ...     30.28   
         14:58        2.79      9.65      7.06      5.60  ...     30.20   
         14:59        2.79      9.65      7.06      5.60  ...     30.20   
         15:00        2.79      9.65      7.06      5.60  ...     30.20   

InstrumentID      SH688210  SH688206  SH688265  SZ301166  SZ301189  SH688227  \
Date     EndTime                                                               
20210104 09:31         NaN       NaN       NaN       NaN       NaN       NaN   
         09:32         NaN       NaN       NaN       NaN       NaN       NaN   
         09:33         NaN       NaN       NaN       NaN       NaN       NaN   
         09:34         NaN       NaN       NaN       NaN       NaN       NaN   
         09:35         NaN       NaN       NaN       NaN       NaN       NaN   
...                    ...       ...       ...       ...       ...       ...   
20211231 14:56       41.48     36.95     69.92    104.04     60.49     32.14   
         14:57       41.51     36.93     70.00    103.70     60.32     32.12   
         14:58       41.48     36.73     70.05    103.20     60.05     32.09   
         14:59       41.48     36.73     70.05    103.30     60.02     32.09   
         15:00       41.48     36.73     70.05    103.30     60.02     32.09   

InstrumentID      SH688236  SZ301127  SH603176  
Date     EndTime                                
20210104 09:31         NaN       NaN       NaN  
         09:32         NaN       NaN       NaN  
         09:33         NaN       NaN       NaN  
         09:34         NaN       NaN       NaN  
         09:35         NaN       NaN       NaN  
...                    ...       ...       ...  
20211231 14:56       27.39     20.23      2.45  
         14:57       27.40     20.30      2.45  
         14:58       27.39     20.25      2.45  
         14:59       27.39     20.19      2.45  
         15:00       27.39     20.19      2.45  

[58320 rows x 4622 columns]

In [6]:
def min_data_cVaR(price:pd.DataFrame):
    def _min_data_cVaR(price_daily:pd.DataFrame):
        return_daily = price_daily.pct_change().fillna(0)
        
        return return_daily.apply(cVaR_series)
    
    return price.groupby(level='Date').progress_apply(_min_data_cVaR)

In [7]:
f1 = min_data_cVaR(open2021)
f1

InstrumentID,SH600000,SH600004,SH600006,SH600007,SH600008,SH600009,SH600010,SH600011,SH600012,SH600015,...,SZ301190,SH688210,SH688206,SH688265,SZ301166,SZ301189,SH688227,SH688236,SZ301127,SH603176
Date,,,,,,,,,,,,,,,,,,,,,
20210104,-0.002489,-0.002604,-0.005117,-0.002957,-0.006110,-0.003075,-0.008547,-0.003470,-0.002889,-0.002897,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
20210105,-0.001675,-0.002426,-0.006855,-0.001922,-0.004014,-0.002728,0.000000,-0.003102,-0.003773,-0.002218,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
20210106,-0.001678,-0.002765,-0.005206,-0.002439,-0.003835,-0.002591,0.000000,-0.002888,-0.004765,-0.001840,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
20210107,-0.002400,-0.003025,-0.006216,-0.003578,-0.003906,-0.002387,-0.008547,-0.003338,-0.003229,-0.002272,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
20210108,-0.002419,-0.002861,-0.005919,-0.002562,-0.003690,-0.002498,0.000000,-0.003215,-0.004016,-0.002475,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20211227,-0.001373,-0.002288,-0.001797,-0.002773,-0.004420,-0.001909,-0.005553,-0.007708,-0.002072,0.000000,...,-0.015806,-0.009663,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
20211228,-0.001472,-0.002883,-0.001591,-0.002156,-0.004972,-0.002129,-0.004163,-0.006616,-0.002195,0.000000,...,-0.005552,-0.008154,-0.016309,-0.010780,-0.019425,-0.019387,0.000000,0.000000,0.000000,0.0
20211229,-0.001870,-0.001966,-0.001919,-0.001751,-0.003259,-0.001980,-0.004474,-0.005607,-0.001945,-0.001789,...,-0.008270,-0.003561,-0.007330,-0.005403,-0.015898,-0.007706,0.000000,0.000000,0.000000,0.0


In [9]:
# f1.to_csv('/mnt/datadisk2/aglv/aglv/lab_aglv/aglv_factor_test/fa_21_23/interday_cvar_neg_0.9_min.csv')

In [10]:
f2 = f1.rolling(5).mean()
f2

InstrumentID,SH600000,SH600004,SH600006,SH600007,SH600008,SH600009,SH600010,SH600011,SH600012,SH600015,...,SZ301190,SH688210,SH688206,SH688265,SZ301166,SZ301189,SH688227,SH688236,SZ301127,SH603176
Date,,,,,,,,,,,,,,,,,,,,,
20210104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210108,-0.002132,-0.002736,-0.005863,-0.002692,-0.004311,-0.002656,-0.003419,-0.003203,-0.003735,-0.002340,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20211227,-0.001336,-0.002060,-0.001699,-0.002815,-0.003498,-0.001617,-0.004913,-0.006316,-0.002158,-0.001448,...,-0.005948,-0.001933,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
20211228,-0.001337,-0.002325,-0.001709,-0.002816,-0.003900,-0.001763,-0.004808,-0.006488,-0.002234,-0.001091,...,-0.007058,-0.003563,-0.003262,-0.002156,-0.003885,-0.003877,0.000000,0.000000,0.000000,0.0
20211229,-0.001432,-0.002201,-0.001782,-0.002683,-0.003908,-0.001794,-0.004902,-0.006430,-0.002074,-0.001092,...,-0.008712,-0.004276,-0.004728,-0.003236,-0.007065,-0.005418,0.000000,0.000000,0.000000,0.0


In [13]:
f1.shift(1).to_csv('/mnt/datadisk2/aglv/aglv/lab_aglv/aglv_factor_test/fa_21_23/interday_cvar_neg_0.9_sft1_min.csv')

In [14]:
open2022 = pd.read_hdf('/mnt/datadisk2/aglv/aglv/lab_aglv/open_2022.h5')
open2022

InstrumentID      SH600000  SH600004  SH600006  SH600007  SH600008  SH600009  \
Date     EndTime                                                               
20220104 09:31        8.54     12.04      6.81     14.40      3.41     46.40   
         09:32        8.54     12.04      6.84     14.39      3.43     46.65   
         09:33        8.55     12.11      6.83     14.38      3.42     46.71   
         09:34        8.54     12.21      6.82     14.36      3.42     46.52   
         09:35        8.55     12.24      6.82     14.37      3.41     46.70   
...                    ...       ...       ...       ...       ...       ...   
20221230 14:56        7.28     15.00      5.57     15.82      2.84     57.69   
         14:57        7.29     15.01      5.57     15.80      2.83     57.72   
         14:58        7.28     15.01      5.57     15.82      2.84     57.73   
         14:59        7.28     15.01      5.57     15.82      2.84     57.73   
         15:00        7.28     15.01      5.57     15.82      2.84     57.73   

InstrumentID      SH600010  SH600011  SH600012  SH600015  ...  SZ301368  \
Date     EndTime                                          ...             
20220104 09:31        2.80      9.92      7.09      5.60  ...       NaN   
         09:32        2.82      9.88      7.09      5.59  ...       NaN   
         09:33        2.82      9.87      7.07      5.59  ...       NaN   
         09:34        2.81      9.84      7.06      5.59  ...       NaN   
         09:35        2.81      9.88      7.06      5.60  ...       NaN   
...                    ...       ...       ...       ...  ...       ...   
20221230 14:56        1.92      7.60      7.31      5.20  ...     19.90   
         14:57        1.92      7.62      7.32      5.20  ...     19.94   
         14:58        1.92      7.61      7.32      5.20  ...     19.95   
         14:59        1.92      7.61      7.32      5.20  ...     19.90   
         15:00        1.92      7.61      7.32      5.20  ...     19.90   

InstrumentID      SZ301369  SZ301377  SZ301379  SZ301380  SZ301388  SZ301389  \
Date     EndTime                                                               
20220104 09:31         NaN       NaN       NaN       NaN       NaN       NaN   
         09:32         NaN       NaN       NaN       NaN       NaN       NaN   
         09:33         NaN       NaN       NaN       NaN       NaN       NaN   
         09:34         NaN       NaN       NaN       NaN       NaN       NaN   
         09:35         NaN       NaN       NaN       NaN       NaN       NaN   
...                    ...       ...       ...       ...       ...       ...   
20221230 14:56       90.27     19.70     25.60     34.82     23.38     18.28   
         14:57       90.27     19.68     25.60     34.87     23.38     18.31   
         14:58       90.17     19.67     25.60     34.89     23.40     18.30   
         14:59       90.17     19.67     25.59     34.89     23.40     18.30   
         15:00       90.17     19.67     25.59     34.89     23.40     18.30   

InstrumentID      SZ301391  SZ301396  SZ301398  
Date     EndTime                                
20220104 09:31         NaN       NaN       NaN  
         09:32         NaN       NaN       NaN  
         09:33         NaN       NaN       NaN  
         09:34         NaN       NaN       NaN  
         09:35         NaN       NaN       NaN  
...                    ...       ...       ...  
20221230 14:56       85.45     34.29     31.10  
         14:57       85.40     34.28     31.10  
         14:58       85.46     34.25     31.13  
         14:59       85.46     34.25     31.13  
         15:00       85.46     34.25     31.13  

[58080 rows x 4947 columns]

In [15]:
f2 = min_data_cVaR(open2022)
f2

InstrumentID,SH600000,SH600004,SH600006,SH600007,SH600008,SH600009,SH600010,SH600011,SH600012,SH600015,...,SZ301368,SZ301369,SZ301377,SZ301379,SZ301380,SZ301388,SZ301389,SZ301391,SZ301396,SZ301398
Date,,,,,,,,,,,,,,,,,,,,,
20220104,-0.001351,-0.002414,-0.001597,-0.001758,-0.003166,-0.001884,-0.005383,-0.006328,-0.001860,-0.001784,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
20220105,-0.001738,-0.003272,-0.001960,-0.002658,-0.003455,-0.002963,-0.004735,-0.005685,-0.004157,-0.001768,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
20220106,-0.001383,-0.003333,-0.001731,-0.003140,-0.003332,-0.002394,-0.004399,-0.005506,-0.002727,-0.001770,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
20220107,-0.001538,-0.002377,-0.001761,-0.002286,-0.003055,-0.002330,-0.004089,-0.004731,-0.002363,-0.001856,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
20220110,-0.001823,-0.003072,-0.001977,-0.002864,-0.002993,-0.002232,-0.004346,-0.003682,-0.001930,-0.001953,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20221226,-0.001926,-0.003213,-0.002112,-0.001891,-0.003597,-0.002076,-0.005263,-0.002744,-0.002481,-0.001953,...,-0.001966,-0.001633,-0.001926,-0.001597,-0.002441,-0.002123,-0.001597,-0.004109,-0.002208,-0.003076
20221227,-0.001393,-0.004508,-0.002353,-0.002238,-0.003559,-0.003293,-0.005181,-0.002681,-0.002079,-0.002030,...,-0.002333,-0.002068,-0.002444,-0.001802,-0.002658,-0.002192,-0.002407,-0.003983,-0.002846,-0.004253
20221228,-0.001651,-0.003081,-0.004029,-0.002394,-0.007092,-0.002061,-0.005102,-0.004060,-0.001918,-0.002253,...,-0.002217,-0.001956,-0.001724,-0.001755,-0.003459,-0.001928,-0.001990,-0.003002,-0.002241,-0.003321


In [16]:
f2.to_csv('/mnt/datadisk2/aglv/aglv/lab_aglv/aglv_factor/fa_21_23/interday_cvar_neg_0.9_2022_min.csv')

In [17]:
f1

InstrumentID,SH600000,SH600004,SH600006,SH600007,SH600008,SH600009,SH600010,SH600011,SH600012,SH600015,...,SZ301190,SH688210,SH688206,SH688265,SZ301166,SZ301189,SH688227,SH688236,SZ301127,SH603176
Date,,,,,,,,,,,,,,,,,,,,,
20210104,-0.002489,-0.002604,-0.005117,-0.002957,-0.006110,-0.003075,-0.008547,-0.003470,-0.002889,-0.002897,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
20210105,-0.001675,-0.002426,-0.006855,-0.001922,-0.004014,-0.002728,0.000000,-0.003102,-0.003773,-0.002218,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
20210106,-0.001678,-0.002765,-0.005206,-0.002439,-0.003835,-0.002591,0.000000,-0.002888,-0.004765,-0.001840,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
20210107,-0.002400,-0.003025,-0.006216,-0.003578,-0.003906,-0.002387,-0.008547,-0.003338,-0.003229,-0.002272,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
20210108,-0.002419,-0.002861,-0.005919,-0.002562,-0.003690,-0.002498,0.000000,-0.003215,-0.004016,-0.002475,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20211227,-0.001373,-0.002288,-0.001797,-0.002773,-0.004420,-0.001909,-0.005553,-0.007708,-0.002072,0.000000,...,-0.015806,-0.009663,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
20211228,-0.001472,-0.002883,-0.001591,-0.002156,-0.004972,-0.002129,-0.004163,-0.006616,-0.002195,0.000000,...,-0.005552,-0.008154,-0.016309,-0.010780,-0.019425,-0.019387,0.000000,0.000000,0.000000,0.0
20211229,-0.001870,-0.001966,-0.001919,-0.001751,-0.003259,-0.001980,-0.004474,-0.005607,-0.001945,-0.001789,...,-0.008270,-0.003561,-0.007330,-0.005403,-0.015898,-0.007706,0.000000,0.000000,0.000000,0.0


In [18]:
f2

InstrumentID,SH600000,SH600004,SH600006,SH600007,SH600008,SH600009,SH600010,SH600011,SH600012,SH600015,...,SZ301368,SZ301369,SZ301377,SZ301379,SZ301380,SZ301388,SZ301389,SZ301391,SZ301396,SZ301398
Date,,,,,,,,,,,,,,,,,,,,,
20220104,-0.001351,-0.002414,-0.001597,-0.001758,-0.003166,-0.001884,-0.005383,-0.006328,-0.001860,-0.001784,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
20220105,-0.001738,-0.003272,-0.001960,-0.002658,-0.003455,-0.002963,-0.004735,-0.005685,-0.004157,-0.001768,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
20220106,-0.001383,-0.003333,-0.001731,-0.003140,-0.003332,-0.002394,-0.004399,-0.005506,-0.002727,-0.001770,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
20220107,-0.001538,-0.002377,-0.001761,-0.002286,-0.003055,-0.002330,-0.004089,-0.004731,-0.002363,-0.001856,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
20220110,-0.001823,-0.003072,-0.001977,-0.002864,-0.002993,-0.002232,-0.004346,-0.003682,-0.001930,-0.001953,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20221226,-0.001926,-0.003213,-0.002112,-0.001891,-0.003597,-0.002076,-0.005263,-0.002744,-0.002481,-0.001953,...,-0.001966,-0.001633,-0.001926,-0.001597,-0.002441,-0.002123,-0.001597,-0.004109,-0.002208,-0.003076
20221227,-0.001393,-0.004508,-0.002353,-0.002238,-0.003559,-0.003293,-0.005181,-0.002681,-0.002079,-0.002030,...,-0.002333,-0.002068,-0.002444,-0.001802,-0.002658,-0.002192,-0.002407,-0.003983,-0.002846,-0.004253
20221228,-0.001651,-0.003081,-0.004029,-0.002394,-0.007092,-0.002061,-0.005102,-0.004060,-0.001918,-0.002253,...,-0.002217,-0.001956,-0.001724,-0.001755,-0.003459,-0.001928,-0.001990,-0.003002,-0.002241,-0.003321


In [20]:
f3 = pd.concat([f1, f2])
f3

,SH600000,SH600004,SH600006,SH600007,SH600008,SH600009,SH600010,SH600011,SH600012,SH600015,...,SZ301368,SZ301369,SZ301377,SZ301379,SZ301380,SZ301388,SZ301389,SZ301391,SZ301396,SZ301398
Date,,,,,,,,,,,,,,,,,,,,,
20210104,-0.002489,-0.002604,-0.005117,-0.002957,-0.006110,-0.003075,-0.008547,-0.003470,-0.002889,-0.002897,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210105,-0.001675,-0.002426,-0.006855,-0.001922,-0.004014,-0.002728,0.000000,-0.003102,-0.003773,-0.002218,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210106,-0.001678,-0.002765,-0.005206,-0.002439,-0.003835,-0.002591,0.000000,-0.002888,-0.004765,-0.001840,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210107,-0.002400,-0.003025,-0.006216,-0.003578,-0.003906,-0.002387,-0.008547,-0.003338,-0.003229,-0.002272,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210108,-0.002419,-0.002861,-0.005919,-0.002562,-0.003690,-0.002498,0.000000,-0.003215,-0.004016,-0.002475,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20221226,-0.001926,-0.003213,-0.002112,-0.001891,-0.003597,-0.002076,-0.005263,-0.002744,-0.002481,-0.001953,...,-0.001966,-0.001633,-0.001926,-0.001597,-0.002441,-0.002123,-0.001597,-0.004109,-0.002208,-0.003076
20221227,-0.001393,-0.004508,-0.002353,-0.002238,-0.003559,-0.003293,-0.005181,-0.002681,-0.002079,-0.002030,...,-0.002333,-0.002068,-0.002444,-0.001802,-0.002658,-0.002192,-0.002407,-0.003983,-0.002846,-0.004253
20221228,-0.001651,-0.003081,-0.004029,-0.002394,-0.007092,-0.002061,-0.005102,-0.004060,-0.001918,-0.002253,...,-0.002217,-0.001956,-0.001724,-0.001755,-0.003459,-0.001928,-0.001990,-0.003002,-0.002241,-0.003321


In [21]:
f3.to_csv('/mnt/datadisk2/aglv/aglv/lab_aglv/aglv_factor/fa_21_23/interday_cvar_neg_0.9_21_22_min.csv')